In [37]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash
import dash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc, html

import plotly.express as px

from dash.dependencies import Input, Output, State
from dash.dash_table.Format import Format, Group, Scheme, Symbol
import base64

import os


# Configure the plotting routines
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser1"
password = "Bingo23459"
shelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))


# data for water rescue dogs
dfwr = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":
             {"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
             "sex_upon_outcome":"Intact Female",
             "age_upon_outcome_in_weeks":{"$gte":26},
             "age_upon_outcome_in_weeks":{"$lte":156}}))

# data for mountatin or wilderness rescue
dfmwr = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":
             {"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Huskey","Rottweiler"]},
             "sex_upon_outcome":"Intact Male",
             "age_upon_outcome_in_weeks":{"$gte":26},
             "age_upon_outcome_in_weeks":{"$lte":156}}))

# data for disastor or individual rescue
dfdit = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":
             {"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},
             "sex_upon_outcome":"Intact Male",
             "age_upon_outcome_in_weeks":{"$gte":20},
             "age_upon_outcome_in_weeks":{"$lte":300}}))


# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
#df.drop(columns=['_id'],inplace=True, axis=1)

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')


#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    #html.Div(id='hidden-div', style={'display':'none'}),
    #html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    #resize logo      
    html.A([html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={
                'height' : '5%',
                'width' : '20%',
                'float' : 'left',
                'position' : 'relative',
                'padding-top' : 0,
                'padding-right' : 0
                })], href='www.snhu.edu'),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard')), style={'font-size': 38}),
    html.Center(html.B(html.H5('Charles Adkins - Module 6 Milestone')), style={'font-size': 24}),
    
    
    html.Center(html.B(html.H5('Types of Search & Rescue Animals')), style={'text-align': 'left', 'font-size': 22}),
    html.Div(className='row', style={'display': 'flex'}, children = [
        
    #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        
        dcc.RadioItems(
            #radio button to filter between different types of rescue dog
            id='filter_type',
            #each "label" is a button the user can press with the correct value for certain
            #search and rescue types
            options=[
                {'label': 'Reset', 'value':'R'},
                {'label': 'Water Rescue', 'value': 'WR'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'MWR'},
                {'label': 'Disaster or Individual Tracking', 'value': 'DIT'}
                
                
            ],
            value='R',
            labelStyle={'display': 'inline-block'},
            style={'padding-left' : '20px', 'font-size': 18}
            )
      
    ]),
    
    
    html.Hr(),
    
 #  columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    
    dash.dash_table.DataTable(
        id='datatable-id',
        columns=[   {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        
        data=df.to_dict('records'),
        
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
        style_as_list_view=True,
    style_cell={'padding': '5px'},
    style_header={
        'backgroundColor': 'white',
        'fontWeight': 'bold'
    },
    style_cell_conditional=[
        {
            'if': {'column_id': c},
            'textAlign': 'left'
        } for c in ['Date', 'Region']
    ]
    
    ),
    html.Br(),
    html.Hr(),
    
    html.Div(className='row', 
         style ={'display' : 'flex'},
            children =[
                html.Div(
                    id='graph-id', 
                    className = 'col s12 m6'
                    ),
                html.Div(
                    id='map-id',
                    className='col s12 m6'
                    )
            ])
])


#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback([Output('datatable-id','data'),
              Output('datatable-id', 'columns')],
              [Input('filter_type', 'value')])

def update_dashboard(filter_type):
    
## FIX ME Add code to filter interactive data table with MongoDB queries
    df = pd.DataFrame.from_records(shelter.read({}))
    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]

    
    if filter_type == 'R':
        df = df
    elif filter_type == 'WR':
        df = dfwr
    elif filter_type == 'MWR':
        df = dfmwr
    elif filter_type == 'DIT':
        df = dfdit
    else:
        df = df

    data = df.to_dict('records')
        
    return (data, columns)

    
    
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]




# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])


def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    
    df = pd.DataFrame.from_dict(viewData)
    # count by breed occurrence
    new_df = df.value_counts()
    new_df.names = 'breed'
    new_df = new_df.to_frame()
    new_df.reset_index(inplace=True)
        
        
    return [
        dcc.Graph(
            figure = px.pie(new_df, names='breed', title='Preferred Animals'),
            style= {'width': '80vh', 'height': '70vh'}
            
            )
        ]



# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])


def update_map(viewData, index):
#FIXME Add in the code for your geolocation chart
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]
    
    

app.run_server(debug=True)

Connection Successful
Dash app running on http://127.0.0.1:19081/
